# Demo 04 - Pandas, Data Tables, and Tidy Data

In this demo we will go over some of the basics of Pandas, so that you have examples of the basic functionality for reference. We then move into working with those data tables, a deep dive on the GroupBy command and how it works, and finish with another deep dive on Melting data and the concepts of Tidy Data.

In [18]:
# first, mount your google drive, change to the course folder, pull latest changes, and change to the lab folder.
# Startup Magic to: (1) Mount Google Drive
# (2) Change to Course Folder
# (3) Pull latest Changes
# (4) Move to the Demo Directory so that the data files are available

from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/cmps3160
!git pull
%cd _demos

ModuleNotFoundError: No module named 'google.colab'

In [53]:
# Includes and Standard Magic...
### Standard Magic and startup initializers.

# Load Numpy
import numpy as np
# Load MatPlotLib
import matplotlib
import matplotlib.pyplot as plt
# Load Pandas
import pandas as pd

# This lets us show plots inline and also save PDF plots if we want them
%matplotlib inline
from matplotlib.backends.backend_pdf import PdfPages
matplotlib.style.use('fivethirtyeight')

# These two things are for Pandas, it widens the notebook and lets us display data easily.
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

# Show a ludicrus number of rows and columns
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

## Basic Pandas Functionality

### Creating Tables

First, let's take a look at some basic Pandas functionality. These are small examples to show the ideas of creating, selecting from, and working with data tables.

In [5]:
s1 = pd.DataFrame([0.3, 0.8, 0.1, 4.0, 1.2], 
                  index = ['a', 'c', 'd', 'e', 'f'], 
                  columns=['data'])

display(s1)

,data
a,0.3
c,0.8
d,0.1
e,4.0
f,1.2


In [6]:
s2 = pd.DataFrame([0.9, 0.1, 4.8, 0.3], 
                  index = ['b', 'c', 'd', 'g'], 
                  columns=['data'])
display(s2)

,data
b,0.9
c,0.1
d,4.8
g,0.3


Re-index can get used to do some funky things or make bigger frames.. this is not common usage but you use reindex to make new entries in an already existing table.  [Doc Page](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.reindex.html)

```Places NA/NaN in locations having no value in the previous index. A new object is produced unless the new index is equivalent to the current one and copy=False.```

In [8]:
# We can use re-index to fill in some gaps if we want..
import string

display(s1.reindex(list(string.ascii_lowercase)[:10]))

# We can also use the 'ffill' method to fill the missing data with the closest data.
display(s1.reindex(list(string.ascii_lowercase)[:10], method='ffill'))

,data
a,0.3
b,NaN
c,0.8
d,0.1
e,4.0
f,1.2
g,NaN
h,NaN
i,NaN
j,NaN


,data
a,0.3
b,0.3
c,0.8
d,0.1
e,4.0
f,1.2
g,1.2
h,1.2
i,1.2
j,1.2


Note that the above operation did not happen in place! so that our s1 table is preserved!

In [10]:
display(s1)

,data
a,0.3
c,0.8
d,0.1
e,4.0
f,1.2


### Filtering, Selecting, and Setting Data.

The first question we might have is how do we get at data in our dataframe? The easiest way is to get a location by it's `index` or `label`. To do this we use the `.loc` command. We can use this command to slice by the index as well!

For more on the difference between loc and iloc see the [10 Mins to Pandas](https://pandas.pydata.org/docs/user_guide/10min.html) documentation.

In [24]:
# Selection by location or loc

s1.loc['f']

data    1.2
Name: f, dtype: float64

In [25]:
s1.loc['a':'d']

,data
a,0.3
c,0.8
d,0.1


In [26]:
s2.loc['b']

data    0.9
Name: b, dtype: float64

We can also use the `iloc` command to get a row by it's **interger position**. 

In [27]:
# Selection by position, iloc
s2.iloc[0]

data    0.9
Name: b, dtype: float64

In [28]:
display(s1.iloc[0:3])

,data
a,0.3
c,0.8
d,0.1


**Gotcha Warning!** Note that loc is inclusive while iloc is not inclusive at the end of the range -- just like indexing into an array!

In [32]:
display(s1)
display(s1.iloc[1:4])


,data
a,0.3
c,0.8
d,0.1
e,4.0
f,1.2


,data
c,0.8
d,0.1
e,4.0


We can use set data in a data frame by using the `at` and `iat` commands.

In [34]:
display(s1)
s1.at['a','data'] = 0
display(s1)

,data
a,0.3
c,0.8
d,0.1
e,4.0
f,1.2


,data
a,0.0
c,0.8
d,0.1
e,4.0
f,1.2


In [37]:
display(s1)
s1.iat[2,0] = 0
display(s1)

,data
a,0.0
c,0.8
d,0.1
e,4.0
f,1.2


,data
a,0.0
c,0.8
d,0.0
e,4.0
f,1.2


Note that both of the above operations happened inplace!! **Why?**

In [39]:
# Let's reset our data frames since we messed them up!

s1 = pd.DataFrame([0.3, 0.8, 0.1, 4.0, 1.2], 
                  index = ['a', 'c', 'd', 'e', 'f'], 
                  columns=['data'])

s2 = pd.DataFrame([0.9, 0.1, 4.8, 0.3], 
                  index = ['b', 'c', 'd', 'g'], 
                  columns=['data'])

In [40]:
# Recall we can also use aggregation operators over the columns

s2['data'].sum()

6.1

As we have seen we can use boolean operations to filter and select data. Recall that these operations return a **view** of the data and not a copy of the dataframe.

In [41]:
s2[s2['data'] > 3]

,data
d,4.8


In [42]:
s1[ (s1['data'] < 1.0) & (s1['data'] > 3.0) ]

,data


In [43]:
s1

,data
a,0.3
c,0.8
d,0.1
e,4.0
f,1.2


### Operations on Data Tables - Broadcasting

When we add constants to a table we get something called **broadcasting** where the operation happens to every element of a table (or column!) You can read more about this essential basic functionality at the [Pandas Documentation Website](https://pandas.pydata.org/docs/user_guide/basics.html)

In [44]:
s1 + 0.5

,data
a,0.8
c,1.3
d,0.6
e,4.5
f,1.7


In [45]:
s2

,data
b,0.9
c,0.1
d,4.8
g,0.3


Note that the above operations did not happen **inplace** so that our tables remain the same as when we declared them! When you manipulate a datatable you must always tell pandas to either save that as a new variable or table, or to do the operation in place.

**Question:** What happens if we add two tables together?

In [46]:
# Note again that these operations did not happen in place!
display(s1)
display(s2)

,data
a,0.3
c,0.8
d,0.1
e,4.0
f,1.2


,data
b,0.9
c,0.1
d,4.8
g,0.3


In [47]:
s1 + s2

,data
a,NaN
b,NaN
c,0.9
d,4.9
e,NaN
f,NaN
g,NaN


In [48]:
s1 - s2

,data
a,NaN
b,NaN
c,0.7
d,-4.7
e,NaN
f,NaN
g,NaN


Note that + is like what we'll learn is an inner join soon but it also adds the numbers together!

You can also apply functions across and down and with lambdas...

In [49]:
frame = pd.DataFrame(np.random.randn(4,3), columns=list('abc'),
                 index=['Utah', 'Ohio', 'Texas','Oregon'])
display(frame)

,a,b,c
Utah,-1.230862,1.032354,-0.711328
Ohio,-1.140589,0.961363,-0.886189
Texas,-1.257380,1.323825,-1.109010
Oregon,1.084507,-0.460657,-0.476700


In [50]:
frame.abs()

,a,b,c
Utah,1.230862,1.032354,0.711328
Ohio,1.140589,0.961363,0.886189
Texas,1.257380,1.323825,1.109010
Oregon,1.084507,0.460657,0.476700


In [51]:
minmax = lambda x: x.max() - x.min()

frame.apply(minmax)

a    2.341887
b    1.784482
c    0.632310
dtype: float64

In [52]:
# Default is axis=0 or per column, can also do per row!
frame.apply(minmax, axis=1)


Utah      2.263216
Ohio      2.101952
Texas     2.581205
Oregon    1.561207
dtype: float64

### Operations on Data Tables - Ranking

Some of the [Pandas Rank](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.rank.html) commands aren't totally obvious...

In [54]:
frame = pd.DataFrame({'a':[0, 1, 0, 1],
                      'b':[4.3, 7, -3, 2],
                   'c':[-2, 5, 8, -2.5]})
display(frame)

,a,b,c
0,0,4.3,-2.0
1,1,7.0,5.0
2,0,-3.0,8.0
3,1,2.0,-2.5


In [55]:
# Default is average but you can do other things!

display(frame.rank(ascending=False))

display(frame.rank(ascending=False, method='first'))

,a,b,c
0,3.5,2.0,3.0
1,1.5,1.0,2.0
2,3.5,4.0,1.0
3,1.5,3.0,4.0


,a,b,c
0,3.0,2.0,3.0
1,1.0,1.0,2.0
2,4.0,4.0,1.0
3,2.0,3.0,4.0


In [ ]:
# Can also sort along rows!

frame.rank(axis=1)

### Importance of using np.nan

Note that we have to import numpy.nan but once we do we can use the groupby and other methods without having to worry about what to do with missing data.

For the example below, what happens when we don't use NaNs in the data table?

In [ ]:
# Make a data frame from a lists
# Try replacing 'XX' with various values..

df = pd.DataFrame({'age':     [12.2, 11.0, 15.6, '--'],
                  'wgt_kg':   [42.3, 40.8, 65.3, 84.2],
                  'hgt_cm':   [145.1, 143.8, 165.3, 185.8],
                  'sex':      ['male', 'female', 'male', 'male'],
                  'country': ['USA', 'AUS', 'EU', 'USA']})
df

In [ ]:
df['age'].describe()

In [ ]:
# Note the type that has been imputed here...
df.dtypes

In [ ]:
df['age'].sum()

In [ ]:
# Need to make sure it's set as number!

df["age"] = pd.to_numeric(df["age"])

In [ ]:
# But first we have to make sure it's a NAN!!
display(df["age"].replace("--", np.nan))

df["age"].replace("--", np.nan,inplace=True)

In [ ]:
df.dtypes

In [ ]:
df["age"].sum()

## The Groupby Command

Below we see what happens in Pandas when we use the groupby command.

In [56]:
df = pd.DataFrame({'age':     [12.2, 11.0, 15.6, 35.1],
                  'wgt_kg':   [42.3, 40.8, 65.3, 84.2],
                  'hgt_cm':   [145.1, 143.8, 165.3, 185.8],
                  'sex':      ['male', 'female', 'male', 'male'],
                  'country': ['USA', 'AUS', 'EU', 'USA']})
df

,age,wgt_kg,hgt_cm,sex,country
0,12.2,42.3,145.1,male,USA
1,11.0,40.8,143.8,female,AUS
2,15.6,65.3,165.3,male,EU
3,35.1,84.2,185.8,male,USA


In [60]:
# This is a little weird, recall that we have lazy evaluation!

df.groupby(['sex'])

In [61]:
# If we describe it then we can force python to not be lazy!

df.groupby('sex').describe()

age                                                      wgt_kg                                                      hgt_cm                                                      
       count       mean        std   min   25%   50%    75%   max  count       mean        std   min   25%   50%    75%   max  count   mean        std    min    25%    50%     75%    max
sex                                                                                                                                                                                       
female   1.0  11.000000        NaN  11.0  11.0  11.0  11.00  11.0    1.0  40.800000        NaN  40.8  40.8  40.8  40.80  40.8    1.0  143.8        NaN  143.8  143.8  143.8  143.80  143.8
male     3.0  20.966667  12.357319  12.2  13.9  15.6  25.35  35.1    3.0  63.933333  20.983406  42.3  53.8  65.3  74.75  84.2    3.0  165.4  20.350184  145.1  155.2  165.3  175.55  185.8

In [62]:
# We can pass operators to groupby to get better results.
df.groupby(['sex']).mean()

,age,wgt_kg,hgt_cm
sex,,,
female,11.000000,40.800000,143.8
male,20.966667,63.933333,165.4


In [63]:
# Can also group by multiple columns
df.groupby(['sex', 'country']).describe()


age                                                      wgt_kg                                                      hgt_cm                                                           
               count   mean        std   min     25%    50%     75%   max  count   mean        std   min     25%    50%     75%   max  count    mean        std    min      25%     50%      75%    max
sex    country                                                                                                                                                                                         
female AUS       1.0  11.00        NaN  11.0  11.000  11.00  11.000  11.0    1.0  40.80        NaN  40.8  40.800  40.80  40.800  40.8    1.0  143.80        NaN  143.8  143.800  143.80  143.800  143.8
male   EU        1.0  15.60        NaN  15.6  15.600  15.60  15.600  15.6    1.0  65.30        NaN  65.3  65.300  65.30  65.300  65.3    1.0  165.30        NaN  165.3  165.300  165.30  165.300  165.3
       USA       2.0  23.65  16.192745  12.2  17.925  23.65  29.375  35.1    2.0  63.25  29.627774  42.3  52.775  63.25  73.725  84.2    2.0  165.45  28.779246  145.1  155.275  165.45  175.625  185.8

In [64]:
# Multi-Index!!
# We can select based on a list of indices
df.groupby(['sex', 'country']).mean().loc['female', 'AUS']

age        11.0
wgt_kg     40.8
hgt_cm    143.8
Name: (female, AUS), dtype: float64

In [65]:
## We can also limit ourselves by just subselecting columns
display(df.groupby(["country"]).count())
display(df.groupby(["country"])[["sex"]].count())

,age,wgt_kg,hgt_cm,sex
country,,,,
AUS,1,1,1,1
EU,1,1,1,1
USA,2,2,2,2


,sex
country,
AUS,1
EU,1
USA,2


In [69]:
# same as...
df['country'].value_counts()

USA    2
EU     1
AUS    1
Name: country, dtype: int64

In [66]:
# This is the same as...
grouped = df.groupby(['country'])

In [70]:
type(grouped)

pandas.core.groupby.generic.DataFrameGroupBy

To get access to a group that we made, use the [get_group() command](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.core.groupby.GroupBy.get_group.html).

We can also get access to the actual tuples and they're types that are generated by the `groupby` commands

In [71]:
grouped.groups

{'AUS': Int64Index([1], dtype='int64'),
 'EU': Int64Index([2], dtype='int64'),
 'USA': Int64Index([0, 3], dtype='int64')}

In [73]:
grouped.mean()

,age,wgt_kg,hgt_cm
country,,,
AUS,11.00,40.80,143.80
EU,15.60,65.30,165.30
USA,23.65,63.25,165.45


In [74]:
grouped.get_group('AUS')

,age,wgt_kg,hgt_cm,sex,country
1,11.0,40.8,143.8,female,AUS


In [75]:
# Or do it fancy?
[display(grouped.get_group(i)) for i in grouped.groups]

,age,wgt_kg,hgt_cm,sex,country
1,11.0,40.8,143.8,female,AUS


,age,wgt_kg,hgt_cm,sex,country
2,15.6,65.3,165.3,male,EU


,age,wgt_kg,hgt_cm,sex,country
0,12.2,42.3,145.1,male,USA
3,35.1,84.2,185.8,male,USA


[None, None, None]

## Joins in Pandas

Let's see a few [joins in Pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.join.html) and how to execute them.

**Note:** We can also use the [merge command in pandas](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) to joins as well.  We'll explore this more fully in the lab.  The key idea is that merge let's us pick columns to do the merge/join while join will always join on the index.

In [ ]:
# Careful here! We have to set the index!

df1 = pd.DataFrame([(1, 'foo', 3), (2, 'bar', 2), (3, 'foo', 4), (4, 'foo', 3)],
                  columns=['ID','A', 'B'])
df1.set_index('ID', inplace=True)
display(df1)

df2 = pd.DataFrame([(1, 1.2), (2, 2.5), (3, 2.3), (5, 8.0)], 
                   columns=['ID','C'])
df2.set_index('ID', inplace=True)
display(df2)

In [ ]:
# Inner join
df1.join(df2, how='inner')

In [ ]:
# Can do inner, outter, left, right.
df1.join(df2, how='right')

If we don't have the same column names we can tell Pandas that we want to join on a specific column.

In [ ]:
df1 = pd.DataFrame([(1, 'foo', 3), (2, 'bar', 2), (3, 'foo', 4), (4, 'foo', 3)],
                  columns=['leftid','A', 'B'])
display(df1)

df2 = pd.DataFrame([(1, 1.2), (2, 2.5), (3, 2.3), (5, 8.0)], 
                   columns=['rightid','C'])
df2.set_index('rightid', inplace=True)
display(df2)

In [ ]:
# Inner join
df1.join(df2, how='inner')
# Is this what we wanted?!?!

In [ ]:
# We could have set the index but we can also tell Pandas to do it for us.
df1.join(df2, how='inner', on='leftid')

## Pandas Merge

Or we can skip all this foolishness by using the [pandas merge](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.merge.html) command...

In [ ]:
# Or we can skip all this foolishness by using the merge command...
df1 = pd.DataFrame([(1, 'foo', 3), (2, 'bar', 2), (3, 'foo', 4), (4, 'foo', 3)],
                  columns=['ID','A', 'B'])
display(df1)

df2 = pd.DataFrame([(1, 1.2), (2, 2.5), (3, 2.3), (5, 8.0)], 
                   columns=['ID','C'])
display(df2)

In [ ]:
x = df1.merge(df2, left_on="ID", right_on="ID", how='inner')

In [ ]:
x

# More Complicated Indicies

To set a **hierarchical index** one can refer to the [documetnation page](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html).

In [ ]:
# Setting a heiararichal index -- The bad way would be to build up tuples as an index.

index = [('California', 2000), ('California', 2010),
         ('New York', 2000), ('New York', 2010),
         ('Texas', 2000), ('Texas', 2010)]
populations = [33871648, 37253956,
               18976457, 19378102,
               20851820, 25145561]
pop = pd.Series(populations, index=index)

In [ ]:
display(pop)

In [ ]:
# Now we can still get slices etc. in a strightforward way.
pop[('California', 2010):('Texas', 2000)]

In [ ]:
# But if we want to get all 2010 data we have to do something like...
pop[[i for i in pop.index if i[1] == 2010]]

In [ ]:
#We can make the data frame as (note it is tidy!)
df = pd.DataFrame([('California', 2000, 33871648),
                   ('California', 2010, 37253956),
                   ('New York', 2000, 18976457),
                   ('New York', 2010, 19378102),
                   ('Texas', 2000, 20851820),
                   ('Texas', 2010, 25145561)],
                  columns=['state', 'year', 'pop'])
df

In [ ]:
df.set_index(['state', 'year'], inplace=True)
df

In [ ]:
# And now we can do cool stuff slicing, but it gets compicated with tuples.
df.loc[('California')]

In [ ]:
df.index

The [df.xs](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.xs.html) command can help with slicing multi-indicies.

In [ ]:
# Thought this can get a bit complicated... 
df.xs(2010, level=1)

# Melting and Tidy Data

In [ ]:
df = pd.read_csv('./data/religon.csv')
df

The [melt command](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.melt.html) does...

```
DataFrame.melt(id_vars=None, value_vars=None, var_name=None, value_name='value', col_level=None, ignore_index=True)

Unpivot a DataFrame from wide to long format, optionally leaving identifiers set.

This function is useful to massage a DataFrame into a format where one or more columns are identifier variables (id_vars), while all other columns, considered measured variables (value_vars), are “unpivoted” to the row axis, leaving just two non-identifier columns, ‘variable’ and ‘value’.
```

So here we want to take the above data frame and make it `grow down` by converting income into a categorical variable.

In [ ]:
f_df = pd.melt(df,
               ["religion"],
               var_name="income",
               value_name="freq")
f_df = f_df.sort_values(by=["income"])
f_df.head(10)

In [ ]:
# We can now use more simple slicing to see the things we need.

f_df[(f_df['religion'] == 'Atheist')].sort_values(by='income')

In [ ]:
f_df[(f_df['income'] == '$50-75k')].sort_values(by='freq', ascending=False)

# A More Complicated Example of Melting...

In [ ]:
b_df = pd.read_csv("./data/billboard.csv")
b_df.head()

This thing is a mess!! What if want to say, graph the positions of a song for a given artist? Then how would we do it?

Again, we are going to melt to make the table `grow down` by keeping the `id_vars` and then melting the rows based on week position to be values.



In [ ]:
# Keep identifier variables
id_vars = ["year",
           "artist.inverted",
           "track",
           "time",
           "genre",
           "date.entered",
           "date.peaked"]

# Melt the rest into week and rank columns
b_df = pd.melt(frame=b_df,
             id_vars=id_vars,
             var_name="week",
             value_name="rank")


In [ ]:
display(b_df.head(20))
b_df.dtypes

In [ ]:
# Let's fix the week thing, it's not good...
b_df["week"] = b_df['week'].str.extract('(\d+)', expand=False).astype(int)

In [ ]:
b_df[:5]

In [ ]:
b_df["week"].unique()

There's a subtle problem here, rank is an int but some are NAN's and we can't have [int-nan's...](https://pandas.pydata.org/pandas-docs/version/0.24/whatsnew/v0.24.0.html#optional-integer-na-support)

In [ ]:
# Why not ints? Gotcha! ?!?!?
b_df["rank"] = b_df["rank"].astype(float)

display(b_df.head(20))

In [ ]:
# We can see if anything is missing...
# Remember axis=1 goes row wise..so we are saying show us any row with missing values...
b_df[b_df.isnull().any(axis=1)]

In [ ]:
# We don't need these so... let's drop them.
b_df = b_df.dropna()

We're going to use the handy [to_datetime](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html) which will get us a date!

In [ ]:
# Create "date" columns
#  date = (date entered chart) + (# of weeks) - (1 week) [fence post problem]
b_df['date'] = pd.to_datetime(b_df['date.entered']) + pd.to_timedelta(b_df['week'], unit='w') - pd.DateOffset(weeks=1) 

In [ ]:
display(b_df[100:110])
b_df.dtypes

In [ ]:
# Ignore now-redundant, messy columns -- same as dropping 
b_df = b_df[["year",
         "artist.inverted",
         "track",
         "time",
         "genre",
         "week",
         "rank",
         "date"]]

b_df = b_df.sort_values(ascending=True, by=["year","artist.inverted","track","week","rank"])

# Keep tidy dataset for future usage
billboard = b_df

billboard.head(10)

In [ ]:
# Now we can see the artists we have...
billboard['artist.inverted'].unique()

In [ ]:
# Easier to visualize!!
b_df[(b_df['artist.inverted'] == 'Jay-Z')]

In [ ]:
b_df[(b_df['artist.inverted'] == 'Jay-Z')]['rank'].plot.hist(bins=100, title="Jay-Z Hits by Week")

In [ ]:
#Scatterplot of JayZ at different songs/days?
b_df[(b_df['artist.inverted'] == 'Jay-Z')].plot.scatter(y='rank', 
                                                        x='date',
                                                        rot=45)

In [ ]:
import seaborn as sns
g = sns.pairplot(x_vars=["date"], 
             y_vars=["rank"], 
             data=b_df[(b_df['artist.inverted'] == 'TLC')], 
             hue="track",
             height=5)

# # Hacky rotation?? -- https://github.com/mwaskom/seaborn/issues/867
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)

# # Invert..
for ax in g.axes.flat:
    ax.invert_yaxis()

In [ ]:
# Or we can get the list of top ranked songs...
b_df[(b_df['rank'] == 1.0)].groupby(['artist.inverted']).count()